In [14]:
import pandas as pd
import numpy as np

#Prep the Population DataFrame by getting rid of unneccessary columns, 
#cleaning the data types, and quierying the information needed.
data_dir = 'C:\\Users\\CAT\\Desktop\\Python'
file_name = data_dir + '\\New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv'
df = pd.read_csv(file_name)

df = df.drop(columns=['FIPS County Code'])
df['Population'] = df['Population'].str.replace(r',', '')
df['Population'] = pd.to_numeric(df['Population'])
df = df.loc[(df['Year']==2010)]
df.dtypes
df.head()

pop = df.groupby(['Borough','Year'],as_index = False).sum().pivot('Borough','Year').fillna(0)
pop

,Population
Year,2010
Borough,
Bronx,1385108
Brooklyn,2504700
Manhattan,1585873
Queens,2230722
Staten Island,468730


In [13]:
#Prep the Crime DataFrame by getting rid of unneccessary columns, 
#cleaning the data types, and quierying the information needed.
file_name2 = data_dir + '\\NYPD_Arrests_Data__Historic_.csv'
dfa =pd.read_csv(file_name2)

dfa = dfa.drop(columns=['ARREST_KEY','X_COORD_CD', 'Latitude', 'Longitude','Y_COORD_CD', 'OFNS_DESC','PD_CD', 'KY_CD', 'LAW_CODE', 'LAW_CAT_CD', 'JURISDICTION_CODE'])
dfa.head(10)

dfa['ARREST_DATE'] = pd.to_datetime(dfa['ARREST_DATE'])
dfa['Year'] = dfa['ARREST_DATE'].dt.year
dfa = dfa.loc[(dfa['Year']==2010)]
dfa = dfa.drop(columns=['Year'])

dfa.loc[(dfa['ARREST_BORO']=='S'), 'ARREST_BORO'] = 'Staten Island'
dfa.loc[(dfa['ARREST_BORO']=='Q'), 'ARREST_BORO'] = 'Queens'
dfa.loc[(dfa['ARREST_BORO']=='M'), 'ARREST_BORO'] = 'Manhattan'
dfa.loc[(dfa['ARREST_BORO']=='B'), 'ARREST_BORO'] = 'Bronx'
dfa.loc[(dfa['ARREST_BORO']=='K'), 'ARREST_BORO'] = 'Brooklyn'

dfa.loc[(dfa['ARREST_BORO']=='Staten Island'), 'Population'] = pop.loc['Staten Island'][0]
dfa.loc[(dfa['ARREST_BORO']=='Queens'), 'Population'] = pop.loc['Queens'][0]
dfa.loc[(dfa['ARREST_BORO']=='Manhattan'), 'Population'] = pop.loc['Manhattan'][0]
dfa.loc[(dfa['ARREST_BORO']=='Bronx'), 'Population'] = pop.loc['Bronx'][0]
dfa.loc[(dfa['ARREST_BORO']=='Brooklyn'), 'Population'] = pop.loc['Brooklyn'][0]


<bound method Series.min of 2531234     468730.0
2531235     468730.0
2531236    2230722.0
2531237    2230722.0
2531238    2230722.0
             ...    
2953551    1385108.0
2953552    2504700.0
2953553    2504700.0
2953554    2504700.0
2953555    2504700.0
Name: Population, Length: 422322, dtype: float64>

In [6]:
def  get_percent_nans(df, column_name):
    colSize = df.shape[0]
    empty = df.loc[df[column_name].isna()]
    emptySize = empty.shape[0]
    return (emptySize/colSize) * 100

for col in dfa.columns:
    print('The \"{}\" column has {:.2f}% empty values'.format(col, get_percent_nans(dfa, col)))

dfa

The "ARREST_DATE" column has 0.00% empty values
The "PD_DESC" column has 0.15% empty values
The "ARREST_BORO" column has 0.00% empty values
The "ARREST_PRECINCT" column has 0.00% empty values
The "AGE_GROUP" column has 0.00% empty values
The "PERP_SEX" column has 0.00% empty values
The "PERP_RACE" column has 0.00% empty values
The "Population" column has 0.00% empty values


,ARREST_DATE,PD_DESC,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,Population
2531234,2010-12-31,"IMPERSONATION 2, PUBLIC SERVANT",Staten Island,122,18-24,M,BLACK,468730.0
2531235,2010-12-31,"MARIJUANA, POSSESSION 4 & 5",Staten Island,120,25-44,F,ASIAN / PACIFIC ISLANDER,468730.0
2531236,2010-12-31,"FORGERY,ETC.,UNCLASSIFIED-FELONY",Queens,113,18-24,F,BLACK,2230722.0
2531237,2010-12-31,"CONTROLLED SUBSTANCE, POSSESSION 7",Queens,101,45-64,M,BLACK,2230722.0
2531238,2010-12-31,"LOITERING,GAMBLING,OTHER",Queens,110,25-44,M,WHITE HISPANIC,2230722.0
...,...,...,...,...,...,...,...,...
2953551,2010-01-01,"BRIBERY,PUBLIC ADMINISTRATION",Bronx,52,18-24,M,BLACK HISPANIC,1385108.0
2953552,2010-01-01,"PUBLIC ADMINISTRATION,UNCLASSIFIED FELONY",Brooklyn,73,18-24,M,BLACK,2504700.0
2953553,2010-01-01,RESISTING ARREST,Brooklyn,75,25-44,M,BLACK,2504700.0
2953554,2010-01-01,"CONTROLLED SUBSTANCE,SALE 3",Brooklyn,73,45-64,F,BLACK,2504700.0


In [4]:
a_boroughs = dfa.groupby('ARREST_BORO')
a_bronx= a_boroughs.get_group('Bronx')
a_staten = a_boroughs.get_group('Staten Island')
a_brooklyn = a_boroughs.get_group('Brooklyn')
a_queens = a_boroughs.get_group('Queens')
a_manhattan = a_boroughs.get_group('Manhattan')

a_boroughs['PD_DESC'].value_counts().head(15)


ARREST_BORO  PD_DESC                                   
Bronx        MARIJUANA, POSSESSION 4 & 5                   14357
             ASSAULT 3                                      8022
             THEFT OF SERVICES, UNCLASSIFIED                7845
             CONTROLLED SUBSTANCE, POSSESSION 7             6467
             TRESPASS 2, CRIMINAL                           4590
             TRESPASS 3, CRIMINAL                           4258
             TRAFFIC,UNCLASSIFIED INFRACTION                4177
             WEAPONS, POSSESSION, ETC                       3484
             CONTROLLED SUBSTANCE,INTENT TO SELL 3          3323
             ROBBERY,UNCLASSIFIED,OPEN AREAS                3092
             ASSAULT 2,1,UNCLASSIFIED                       3055
             CONTROLLED SUBSTANCE,SALE 3                    2778
             LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED     1984
             STOLEN PROPERTY 3,POSSESSION                   1964
             MARIJUANA, SALE 4 & 5

In [15]:
dfa.size


3378576